# Import

In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, regularizers, evaluators, trainers, datasets, distributions

# DataSet

In [2]:
dataset = datasets.Yelp()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.implicit_feedback_data(tail_threshold=20)

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)

# Evaluator

In [3]:
# k
ks = [10, 50, 100]

# Accuracy
score_function_dict = {
    # "nDCG" : evaluators.ndcg,
    # "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
accuracy_evaluator = evaluators.AccuracyEvaluator(test_set, score_function_dict, ks)

# Unpopularity
unpopularity_evaluator = evaluators.UnpopularityEvaluator(test_set, ks)

# F1-Score (Recall-Unpopularity)
f1_score_evaluator = evaluators.F1ScoreEvaluator(test_set, ks)

# Coverage
coverage_evaluator = evaluators.CoverageEvaluator(test_set, ks)

# Hubness
hubness_evaluator = evaluators.HubnessEvaluator(test_set, ks)

# Sampler

In [4]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

# Model

In [5]:
# Hyperparameters
lr = 1e-3
n_dim = 50
n_batch = 256
n_epoch = 50
valid_per_epoch = 10
n_item_sample = 30
n_user_sample = 30

# distributiuons
dist1 = distributions.Empirical()
dist2 = distributions.Gaussian()
dist3 = distributions.Gamma()

# models
model0 = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
model1 = models.MutualProximityCML(n_user, n_item, dist1, n_dim).to(device)
model2 = models.MutualProximityCML(n_user, n_item, dist2, n_dim).to(device)
model3 = models.MutualProximityCML(n_user, n_item, dist3, n_dim).to(device)

# learning late optimizer
optimizer0 = optim.Adam(model0.parameters(), lr=lr)
optimizer1 = optim.Adam(model1.parameters(), lr=lr)
optimizer2 = optim.Adam(model2.parameters(), lr=lr)
optimizer3 = optim.Adam(model3.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer0 = trainers.BaseTrainer(model0, optimizer0, criterion, sampler)
trainer1 = trainers.BaseTrainer(model1, optimizer1, criterion, sampler)
trainer2 = trainers.BaseTrainer(model2, optimizer2, criterion, sampler)
trainer3 = trainers.BaseTrainer(model3, optimizer3, criterion, sampler)

# Accuracy

In [6]:
# train0
trainer0.fit(n_batch, n_epoch, accuracy_evaluator, valid_per_epoch)

100%|█████████████████████████████████████████████████████████████████████| 943/943 [00:12<00:00, 76.93it/s]


In [7]:
# train3
trainer3.fit(n_batch, n_epoch, accuracy_evaluator, valid_per_epoch)

100%|█████████████████████████████████████████████████████████████████████| 943/943 [01:20<00:00, 11.69it/s]


In [8]:
# CML
trainer0.valid_scores

,Recall@10,Recall@50,Recall@100,epoch,losses
0,0.005954,0.031255,0.061262,0,NaN
0,0.196918,0.479253,0.617527,10,0.224547
0,0.236741,0.554036,0.699025,20,0.185461
0,0.247679,0.564134,0.709211,30,0.178184
0,0.251454,0.574031,0.709769,40,0.176087
0,0.252976,0.564754,0.710621,50,0.175718


In [9]:
# MPCML
trainer3.valid_scores

,Recall@10,Recall@50,Recall@100,epoch,losses
0,0.006406,0.031822,0.064468,0,NaN
0,0.091513,0.314716,0.480415,10,0.224150
0,0.120586,0.381543,0.549563,20,0.186273
0,0.117787,0.387715,0.567447,30,0.179175
0,0.126252,0.399485,0.574828,40,0.177601
0,0.126010,0.394598,0.571777,50,0.175518


# Unpopularity

In [6]:
# train0
trainer0.fit(n_batch, n_epoch, unpopularity_evaluator, valid_per_epoch)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 943/943 [00:13<00:00, 69.31it/s]


In [7]:
# train3
trainer3.fit(n_batch, n_epoch, unpopularity_evaluator, valid_per_epoch)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 943/943 [01:24<00:00, 11.16it/s]


In [8]:
# CML
trainer0.valid_scores

,Unpopularity@10,Unpopularity@50,Unpopularity@100,epoch,losses
0,0.006281,0.031371,0.062552,0,NaN
0,0.000023,0.000608,0.004014,10,0.223792
0,0.000124,0.002934,0.011536,20,0.185561
0,0.000163,0.003434,0.012627,30,0.178603
0,0.000172,0.003515,0.012809,40,0.177455
0,0.000199,0.003706,0.013303,50,0.176095


In [9]:
# MPCML
trainer3.valid_scores

,Unpopularity@10,Unpopularity@50,Unpopularity@100,epoch,losses
0,0.006288,0.031349,0.062548,0,NaN
0,0.002382,0.015721,0.036192,10,0.224363
0,0.002672,0.017033,0.038904,20,0.185202
0,0.002648,0.017168,0.039306,30,0.179584
0,0.002651,0.017155,0.039290,40,0.175595
0,0.002659,0.017327,0.039557,50,0.175050


# F1-Force

In [6]:
# train0
trainer0.fit(n_batch, n_epoch, f1_score_evaluator, valid_per_epoch)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 943/943 [00:16<00:00, 57.25it/s]


In [7]:
# train3
trainer3.fit(n_batch, n_epoch, f1_score_evaluator, valid_per_epoch)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 943/943 [01:47<00:00,  8.80it/s]


In [8]:
# CML
trainer0.valid_scores

,F1-Score@10,F1-Score@50,F1-Score@100,epoch,losses
0,0.000831,0.014719,0.039676,0,NaN
0,0.000027,0.001278,0.008496,10,0.223168
0,0.000205,0.005538,0.021902,20,0.185586
0,0.000284,0.006709,0.024035,30,0.180622
0,0.000310,0.006750,0.024082,40,0.177439
0,0.000304,0.006892,0.024555,50,0.175953


In [9]:
# MPCML
trainer3.valid_scores

,F1-Score@10,F1-Score@50,F1-Score@100,epoch,losses
0,0.000867,0.014002,0.038383,0,NaN
0,0.001985,0.022243,0.057425,10,0.226947
0,0.002767,0.027332,0.067365,20,0.187260
0,0.002999,0.029062,0.068615,30,0.179385
0,0.003123,0.029503,0.069295,40,0.176829
0,0.003081,0.029455,0.069231,50,0.177244


# Coverage

In [10]:
# train0
trainer0.fit(n_batch, n_epoch, coverage_evaluator, valid_per_epoch)

100%|████████████████████████████████████████████████████████████████████| 943/943 [00:05<00:00, 163.98it/s]


In [11]:
# train3
trainer3.fit(n_batch, n_epoch, coverage_evaluator, valid_per_epoch)

100%|█████████████████████████████████████████████████████████████████████| 943/943 [01:15<00:00, 12.56it/s]


In [12]:
# CML
trainer0.valid_scores

,coverage@10,coverage@50,coverage@100,epoch,losses
0,0.444114,0.689655,0.839477,0,NaN
0,0.443520,0.699168,0.842449,10,0.174707
0,0.438763,0.696790,0.836504,20,0.174668
0,0.447087,0.705113,0.852556,30,0.174080
0,0.445898,0.707491,0.859096,40,0.174076
0,0.447087,0.715220,0.860285,50,0.172866


In [13]:
# MPCML
trainer3.valid_scores

,coverage@10,coverage@50,coverage@100,epoch,losses
0,0.705707,0.885256,0.947087,0,NaN
0,0.712842,0.897741,0.945898,10,0.174676
0,0.710464,0.898335,0.948870,20,0.175411
0,0.702735,0.907848,0.950654,30,0.173591
0,0.708086,0.906659,0.947681,40,0.173612
0,0.706897,0.897146,0.946492,50,0.173462


# Hubness 

In [18]:
# train0
trainer0.fit(n_batch, n_epoch, hubness_evaluator, valid_per_epoch)

100%|████████████████████████████████████████████████████████████████████| 943/943 [00:06<00:00, 155.59it/s]


In [19]:
# train3
trainer3.fit(n_batch, n_epoch, hubness_evaluator, valid_per_epoch)

100%|█████████████████████████████████████████████████████████████████████| 943/943 [01:26<00:00, 10.90it/s]


In [20]:
# CML
trainer0.valid_scores

,hubness@10,hubness@50,hubness@100,epoch,losses
0,2.993997,2.472236,1.904482,0,NaN
0,2.994896,2.433877,1.835709,10,0.174507
0,2.954879,2.468611,1.895725,20,0.172921
0,3.337296,2.345126,1.821007,30,0.173450
0,3.213979,2.637973,1.912664,40,0.174304
0,3.147580,2.529649,1.866664,50,0.173165


In [21]:
# MPCML
trainer3.valid_scores

,hubness@10,hubness@50,hubness@100,epoch,losses
0,1.155360,0.842149,0.663550,0,NaN
0,1.135682,0.854684,0.677147,10,0.172490
0,1.099152,0.830545,0.675085,20,0.173634
0,1.080685,0.862235,0.653784,30,0.174280
0,1.185578,0.862683,0.672255,40,0.174261
0,1.082539,0.867734,0.663411,50,0.173430
